In [4]:
%load_ext autoreload
%autoreload 1

%aimport sheet
%aimport results_comparer
%aimport plot

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
sheet.initial_display()

Button(description='Sort by AP', style=ButtonStyle())

Button(description='Sort by FPRat95', style=ButtonStyle())

Output(outputs=({'output_type': 'display_data', 'data': {'text/plain': '<IPython.core.display.HTML object>', '…

In [6]:
results_comparer.display_controls()